07-24-2019. remove duplicate affiliations based on fuzzy string

07-25-2019. load the calculated scores to find out those above a certain threhold)

---

## check out duplicate affiliation

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sqlalchemy import create_engine

In [3]:
import fuzzywuzzy
from fuzzywuzzy import fuzz

/Users/yihuiwang/Envs/ivy/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
from itertools import combinations
from joblib import Parallel, delayed

In [5]:
#count cpu
#import psutil
#print(psutil.cpu_count())
import multiprocessing
print(multiprocessing.cpu_count())

4


In [6]:
username = 'yihuiwang'
#username = 'zhiyzuo'
#------
#username = 'postgres'
pwd = ''
engine_str = 'postgresql://%s:%s@localhost:5432/ais'%(username,pwd)
engine_str

'postgresql://yihuiwang:@localhost:5432/ais'

In [7]:
engine = create_engine(engine_str)

In [8]:
sql_cmd = "select * from aff_info"
df = pd.read_sql(sql = sql_cmd, con = engine)

In [9]:
engine.dispose()

In [11]:
df.shape

(9099, 10)

In [12]:
df_aff = df['affiliation-name'].values.copy()

In [13]:
df_aff[:5]

array(['McGill University', 'Xidian University',
       'Brunel University London',
       'University of Massachusetts Dartmouth',
       'University of Maryland, Baltimore County'], dtype=object)

### generating conbinations

In [13]:
#list0 = df_aff.tolist()

In [14]:
list_aff_pair = list(combinations(df_aff, 2))

In [15]:
df.shape[0]*(df.shape[0]-1)/2

41391351.0

In [16]:
len(list_aff_pair)

41391351

### split into multiple sub-arrays (e.g. 2100)

In [17]:
num_splits = 10000
list_split = np.array_split(list_aff_pair, num_splits)

In [18]:
list_split[0].shape, len(list_split)

((4140, 2), 10000)

the following code does not work
```python
def fun(x):
    aff_pair_score = pd.DataFrame(columns=('Affiliation1', 'Affiliation2',
                                           'Ratio', 'Token_Sort_Ratio'))
    for entry in x:
        str1 = entry[0]
        str2 = entry[1]
        Ratio = fuzz.ratio(str1.lower(),str2.lower())
        Token_Sort_Ratio = fuzz.token_sort_ratio(str1, str2)
        aff_pair_score = aff_pair_score.append(pd.DataFrame({'Affiliation1':[str1],'Affiliation2':[str2],'Ratio':[Ratio],'Token_Sort_Ratio':[Token_Sort_Ratio]}),ignore_index=True)
    return aff_pair_score
```

In [19]:
def wrapper(str1, str2):
    Ratio = fuzz.ratio(str1.lower(),str2.lower())
    Token_Sort_Ratio = fuzz.token_sort_ratio(str1, str2)
    return {'Affiliation1': str1,
            'Affiliation2': str2,
            'Ratio': Ratio,
            'Token_Sort_Ratio': Token_Sort_Ratio}

In [20]:
list_split[0][0]

array(['McGill University', 'Xidian University'], dtype='<U118')

some random examples to verify the `fuzzywuzzy` method 

In [21]:
wrapper(*['McGill University', 'Xidian University'])

{'Affiliation1': 'McGill University',
 'Affiliation2': 'Xidian University',
 'Ratio': 71,
 'Token_Sort_Ratio': 59}

In [22]:
wrapper(*['University of Maryland, Baltimore County', 'University of Maryland at Baltimore County'])

{'Affiliation1': 'University of Maryland, Baltimore County',
 'Affiliation2': 'University of Maryland at Baltimore County',
 'Ratio': 95,
 'Token_Sort_Ratio': 96}

---

## run 

### test with a subset

In [23]:
len(list_split)

10000

In [24]:
num_test = 100
y = Parallel(n_jobs=3)(delayed(wrapper)(*item) for l_list in list_split[:num_test] for item in l_list)

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(y).query('Ratio>80')

### run for all

In [33]:
y = Parallel(n_jobs=6)(delayed(wrapper)(*item) for l_list in list_split for item in l_list)

To avoid memory error, i may need to split the data into chunks

In [38]:
len(y)

41391351

In [47]:
i = 0
num_steps = 10000
for start_i in np.arange(0, len(y), num_steps).tolist()+[len(y)]: 
    stop_i = min(start_i + num_steps, len(y))
    pd.DataFrame(y[start_i:stop_i]).to_csv('data/%i_fuzzscores.csv'%i, index=False)
    i += 1

--- 

## read results

In [8]:
import os

In [9]:
base_dir = 'data/'
file_l = [base_dir + f for f in os.listdir(base_dir)]
len(file_l)

4141

### test with a threshold

In [14]:
thres = 90

In [54]:
i = 4140
fname = file_l[i]
fname

'data/3409_fuzzscores.csv'

In [55]:
tmp_df = pd.read_csv(fname)
tmp_df.head(2)

,Affiliation1,Affiliation2,Ratio,Token_Sort_Ratio
0,U.S. Army Artificial Intelligence,University of Ceorgia,30,30
1,U.S. Army Artificial Intelligence,"A-Star, Institute for Infocomm Research",36,43


In [56]:
tmp_df.query('Ratio>@thres or Token_Sort_Ratio>@thres')

,Affiliation1,Affiliation2,Ratio,Token_Sort_Ratio


### entries above 90 for ratio or token_sort_ratio

In [18]:
thres = 90

In [50]:
df_aff_90_1 = pd.DataFrame()
df_tmp = pd.DataFrame()
for i in range(2140):
    fname = file_l[i]
    #print(fname,i)
    tmp_df = pd.read_csv(fname)
    #tmp_df.head(2)
    df_tmp = tmp_df.query('Ratio>@thres or Token_Sort_Ratio>@thres')
    df_aff_90_1 = df_aff_90_1.append(df_tmp)
    #print(fname)
    i += 1

data/3887_fuzzscores.csv 0
data/3892_fuzzscores.csv 1
data/1663_fuzzscores.csv 2
data/1676_fuzzscores.csv 3
data/2876_fuzzscores.csv 4
data/2863_fuzzscores.csv 5
data/606_fuzzscores.csv 6
data/613_fuzzscores.csv 7
data/3122_fuzzscores.csv 8
data/3137_fuzzscores.csv 9
data/1277_fuzzscores.csv 10
data/1262_fuzzscores.csv 11
data/212_fuzzscores.csv 12
data/207_fuzzscores.csv 13
data/3536_fuzzscores.csv 14
data/3523_fuzzscores.csv 15
data/2203_fuzzscores.csv 16
data/2216_fuzzscores.csv 17
data/867_fuzzscores.csv 18
data/872_fuzzscores.csv 19
data/2617_fuzzscores.csv 20
data/2602_fuzzscores.csv 21
data/1802_fuzzscores.csv 22
data/1817_fuzzscores.csv 23
data/4038_fuzzscores.csv 24
data/1775_fuzzscores.csv 25
data/1760_fuzzscores.csv 26
data/3991_fuzzscores.csv 27
data/3984_fuzzscores.csv 28
data/2298_fuzzscores.csv 29
data/558_fuzzscores.csv 30
data/3269_fuzzscores.csv 31
data/2960_fuzzscores.csv 32
data/2975_fuzzscores.csv 33
data/710_fuzzscores.csv 34
data/705_fuzzscores.csv 35
data/1528_f

data/1767_fuzzscores.csv 294
data/1772_fuzzscores.csv 295
data/1810_fuzzscores.csv 296
data/1805_fuzzscores.csv 297
data/2605_fuzzscores.csv 298
data/2610_fuzzscores.csv 299
data/875_fuzzscores.csv 300
data/860_fuzzscores.csv 301
data/398_fuzzscores.csv 302
data/2458_fuzzscores.csv 303
data/2211_fuzzscores.csv 304
data/2204_fuzzscores.csv 305
data/3918_fuzzscores.csv 306
data/799_fuzzscores.csv 307
data/2059_fuzzscores.csv 308
data/3524_fuzzscores.csv 309
data/3531_fuzzscores.csv 310
data/1038_fuzzscores.csv 311
data/200_fuzzscores.csv 312
data/215_fuzzscores.csv 313
data/3779_fuzzscores.csv 314
data/1988_fuzzscores.csv 315
data/2788_fuzzscores.csv 316
data/1265_fuzzscores.csv 317
data/1270_fuzzscores.csv 318
data/3130_fuzzscores.csv 319
data/3125_fuzzscores.csv 320
data/614_fuzzscores.csv 321
data/601_fuzzscores.csv 322
data/1439_fuzzscores.csv 323
data/3378_fuzzscores.csv 324
data/2864_fuzzscores.csv 325
data/2871_fuzzscores.csv 326
data/1671_fuzzscores.csv 327
data/1664_fuzzscores.c

data/2269_fuzzscores.csv 582
data/3298_fuzzscores.csv 583
data/2991_fuzzscores.csv 584
data/2984_fuzzscores.csv 585
data/1208_fuzzscores.csv 586
data/3701_fuzzscores.csv 587
data/3714_fuzzscores.csv 588
data/278_fuzzscores.csv 589
data/1040_fuzzscores.csv 590
data/1055_fuzzscores.csv 591
data/980_fuzzscores.csv 592
data/995_fuzzscores.csv 593
data/3549_fuzzscores.csv 594
data/431_fuzzscores.csv 595
data/424_fuzzscores.csv 596
data/1609_fuzzscores.csv 597
data/2809_fuzzscores.csv 598
data/3315_fuzzscores.csv 599
data/3300_fuzzscores.csv 600
data/1454_fuzzscores.csv 601
data/1441_fuzzscores.csv 602
data/679_fuzzscores.csv 603
data/3148_fuzzscores.csv 604
data/4120_fuzzscores.csv 605
data/4135_fuzzscores.csv 606
data/2552_fuzzscores.csv 607
data/2547_fuzzscores.csv 608
data/287_fuzzscores.csv 609
data/292_fuzzscores.csv 610
data/3812_fuzzscores.csv 611
data/3807_fuzzscores.csv 612
data/2146_fuzzscores.csv 613
data/2153_fuzzscores.csv 614
data/693_fuzzscores.csv 615
data/686_fuzzscores.csv

data/2387_fuzzscores.csv 876
data/1422_fuzzscores.csv 877
data/1437_fuzzscores.csv 878
data/1993_fuzzscores.csv 879
data/1986_fuzzscores.csv 880
data/3777_fuzzscores.csv 881
data/3762_fuzzscores.csv 882
data/2786_fuzzscores.csv 883
data/2793_fuzzscores.csv 884
data/1036_fuzzscores.csv 885
data/1023_fuzzscores.csv 886
data/2042_fuzzscores.csv 887
data/2057_fuzzscores.csv 888
data/797_fuzzscores.csv 889
data/782_fuzzscores.csv 890
data/3916_fuzzscores.csv 891
data/3903_fuzzscores.csv 892
data/2456_fuzzscores.csv 893
data/2443_fuzzscores.csv 894
data/383_fuzzscores.csv 895
data/396_fuzzscores.csv 896
data/4024_fuzzscores.csv 897
data/4031_fuzzscores.csv 898
data/2969_fuzzscores.csv 899
data/3275_fuzzscores.csv 900
data/3260_fuzzscores.csv 901
data/2284_fuzzscores.csv 902
data/2291_fuzzscores.csv 903
data/551_fuzzscores.csv 904
data/544_fuzzscores.csv 905
data/1769_fuzzscores.csv 906
data/3998_fuzzscores.csv 907
data/3028_fuzzscores.csv 908
data/1534_fuzzscores.csv 909
data/1521_fuzzscores

data/522_fuzzscores.csv 1159
data/3213_fuzzscores.csv 1160
data/3206_fuzzscores.csv 1161
data/1552_fuzzscores.csv 1162
data/1547_fuzzscores.csv 1163
data/123_fuzzscores.csv 1164
data/136_fuzzscores.csv 1165
data/36_fuzzscores.csv 1166
data/23_fuzzscores.csv 1167
data/3607_fuzzscores.csv 1168
data/3612_fuzzscores.csv 1169
data/1146_fuzzscores.csv 1170
data/1153_fuzzscores.csv 1171
data/886_fuzzscores.csv 1172
data/893_fuzzscores.csv 1173
data/6_fuzzscores.csv 1174
data/2024_fuzzscores.csv 1175
data/2031_fuzzscores.csv 1176
data/2279_fuzzscores.csv 1177
data/3970_fuzzscores.csv 1178
data/3965_fuzzscores.csv 1179
data/1794_fuzzscores.csv 1180
data/1781_fuzzscores.csv 1181
data/2981_fuzzscores.csv 1182
data/2994_fuzzscores.csv 1183
data/3288_fuzzscores.csv 1184
data/2430_fuzzscores.csv 1185
data/2425_fuzzscores.csv 1186
data/808_fuzzscores.csv 1187
data/1380_fuzzscores.csv 1188
data/1395_fuzzscores.csv 1189
data/2678_fuzzscores.csv 1190
data/1878_fuzzscores.csv 1191
data/3689_fuzzscores.cs

data/1154_fuzzscores.csv 1442
data/1141_fuzzscores.csv 1443
data/379_fuzzscores.csv 1444
data/24_fuzzscores.csv 1445
data/31_fuzzscores.csv 1446
data/3615_fuzzscores.csv 1447
data/3600_fuzzscores.csv 1448
data/131_fuzzscores.csv 1449
data/124_fuzzscores.csv 1450
data/1309_fuzzscores.csv 1451
data/3049_fuzzscores.csv 1452
data/778_fuzzscores.csv 1453
data/1540_fuzzscores.csv 1454
data/1555_fuzzscores.csv 1455
data/3201_fuzzscores.csv 1456
data/3214_fuzzscores.csv 1457
data/2908_fuzzscores.csv 1458
data/1708_fuzzscores.csv 1459
data/525_fuzzscores.csv 1460
data/530_fuzzscores.csv 1461
data/1969_fuzzscores.csv 1462
data/3798_fuzzscores.csv 1463
data/1284_fuzzscores.csv 1464
data/1291_fuzzscores.csv 1465
data/2769_fuzzscores.csv 1466
data/919_fuzzscores.csv 1467
data/2534_fuzzscores.csv 1468
data/2521_fuzzscores.csv 1469
data/2885_fuzzscores.csv 1470
data/2890_fuzzscores.csv 1471
data/3399_fuzzscores.csv 1472
data/2368_fuzzscores.csv 1473
data/3874_fuzzscores.csv 1474
data/3861_fuzzscores.

data/2970_fuzzscores.csv 1723
data/2965_fuzzscores.csv 1724
data/3279_fuzzscores.csv 1725
data/1538_fuzzscores.csv 1726
data/700_fuzzscores.csv 1727
data/715_fuzzscores.csv 1728
data/3024_fuzzscores.csv 1729
data/3031_fuzzscores.csv 1730
data/299_fuzzscores.csv 1731
data/2559_fuzzscores.csv 1732
data/961_fuzzscores.csv 1733
data/974_fuzzscores.csv 1734
data/2711_fuzzscores.csv 1735
data/2704_fuzzscores.csv 1736
data/1904_fuzzscores.csv 1737
data/1911_fuzzscores.csv 1738
data/698_fuzzscores.csv 1739
data/2158_fuzzscores.csv 1740
data/3819_fuzzscores.csv 1741
data/2305_fuzzscores.csv 1742
data/2310_fuzzscores.csv 1743
data/2150_fuzzscores.csv 1744
data/2145_fuzzscores.csv 1745
data/685_fuzzscores.csv 1746
data/690_fuzzscores.csv 1747
data/3804_fuzzscores.csv 1748
data/3811_fuzzscores.csv 1749
data/2318_fuzzscores.csv 1750
data/2544_fuzzscores.csv 1751
data/2551_fuzzscores.csv 1752
data/291_fuzzscores.csv 1753
data/284_fuzzscores.csv 1754
data/969_fuzzscores.csv 1755
data/2719_fuzzscores.

data/3769_fuzzscores.csv 2005
data/1998_fuzzscores.csv 2006
data/1275_fuzzscores.csv 2007
data/1260_fuzzscores.csv 2008
data/2798_fuzzscores.csv 2009
data/3529_fuzzscores.csv 2010
data/218_fuzzscores.csv 2011
data/1020_fuzzscores.csv 2012
data/1035_fuzzscores.csv 2013
data/1985_fuzzscores.csv 2014
data/1990_fuzzscores.csv 2015
data/3761_fuzzscores.csv 2016
data/3774_fuzzscores.csv 2017
data/1268_fuzzscores.csv 2018
data/2790_fuzzscores.csv 2019
data/2785_fuzzscores.csv 2020
data/3128_fuzzscores.csv 2021
data/1434_fuzzscores.csv 2022
data/1421_fuzzscores.csv 2023
data/619_fuzzscores.csv 2024
data/2869_fuzzscores.csv 2025
data/3375_fuzzscores.csv 2026
data/3360_fuzzscores.csv 2027
data/451_fuzzscores.csv 2028
data/444_fuzzscores.csv 2029
data/2384_fuzzscores.csv 2030
data/2391_fuzzscores.csv 2031
data/3898_fuzzscores.csv 2032
data/1669_fuzzscores.csv 2033
data/1808_fuzzscores.csv 2034
data/4032_fuzzscores.csv 2035
data/4027_fuzzscores.csv 2036
data/2608_fuzzscores.csv 2037
data/878_fuzzs

In [59]:
df_aff_90_2 = pd.DataFrame()
df_tmp = pd.DataFrame()
for i in range(2141,4140):
    fname = file_l[i]
    tmp_df = pd.read_csv(fname)
    #tmp_df.head(2)
    df_tmp = tmp_df.query('Ratio>@thres or Token_Sort_Ratio>@thres')
    df_aff_90_2 = df_aff_90_2.append(df_tmp)
    #print(fname)
    i += 1

In [63]:
df_aff_90 = pd.concat([df_aff_90_1,df_aff_90_2])

In [64]:
df_aff_90

,Affiliation1,Affiliation2,Ratio,Token_Sort_Ratio
2594,University of Muünster,University of Miinster,91,93
2857,University of Muünster,University of Mönster,93,98
6887,The Bahá'í Institute for Higher Education (BIHE),Bahai Institute for Higher Education (BIHE,89,93
4080,Euro-Arab Management School,Euro-Arab Management School,100,100
5674,Clarkson University,Clarion University,92,92
6313,Clarkson University,Clark University,91,91
7798,Findwise AB,Findwise AB,100,100
8901,Findwise AB,Findwise AB,100,100
3936,Vector Consulting Services GmbH,Vector Consulting Services,91,91
5854,Vector Consulting Services GmbH,Vector Consulting Services,91,91


In [93]:
import csv
#df.to_csv('./a.csv', sep=',', header=True, index=True)
df_aff_90.to_csv('duplicate_entries.csv')

### Duplicate entries

In [89]:
df_same = df_aff_90[df_aff_90['Token_Sort_Ratio'] ==100]

In [90]:
df_same

,Affiliation1,Affiliation2,Ratio,Token_Sort_Ratio
4080,Euro-Arab Management School,Euro-Arab Management School,100,100
7798,Findwise AB,Findwise AB,100,100
8901,Findwise AB,Findwise AB,100,100
7448,Deloitte Consulting,Deloitte Consulting,100,100
8692,Deloitte Consulting,Deloitte Consulting,100,100
763,Yuhan College,Yuhan College,100,100
817,Yuhan College,Yuhan College,100,100
1158,Yuhan College,Yuhan College,100,100
7815,Yuhan College,Yuhan College,100,100
5589,Ernst and Young,Ernst and Young,100,100


In [14]:
df_dup = pd.read_excel('duplicate_entries_final.xlsx')

In [15]:
df_aff

array(['McGill University', 'Xidian University',
       'Brunel University London', ..., 'Korea District Heating Corp.',
       'Advanced Technology Center', 'Theseus Institute'], dtype=object)

In [16]:
df_dup

,Affiliation1,Affiliation2,Aff
0,"A.O. Smith Data Systems, Inc.",A. O. Smith Data Systems,"A.O. Smith Data Systems, Inc."
1,Accenture,Accenture,Accenture
2,ACM Education Board,ACM Education Board,ACM Education Board
3,Addis Ababa University,Addis Ababa University,Addis Ababa University
4,"Advanced Brain Monitoring, Inc.",Advanced Brain Monitoring,"Advanced Brain Monitoring, Inc."
5,AFAS Software,AFAS Sofware,AFAS Software
6,Al Akhawayn University,Al Akhawayan University,Al Akhawayn University
7,Alabama A and M University,Alabama AandM University,Alabama A and M University
8,ALBA Graduate Business School,ALBA Graduate Business School,ALBA Graduate Business School
9,"ALDI, Inc.","ALDI, Inc.","ALDI, Inc."


In [47]:
sql_cmd = "select * from aff_info"
df = pd.read_sql(sql = sql_cmd, con = engine)

In [51]:
df_affname = df.iloc[:,1:3]

In [53]:
df_affname

,aff_id,affiliation-name
0,60002494,McGill University
1,60025578,Xidian University
2,60020623,Brunel University London
3,60009060,University of Massachusetts Dartmouth
4,60024997,"University of Maryland, Baltimore County"
5,60013959,University of Florida
6,60021508,University of Notre Dame
7,60080184,University of Agder
8,60027786,Universität St. Gallen
9,60024063,University of Massachusetts Boston


### select weird affiliation and count author

In [72]:
sql_cmd = "select * from author_affiliation_pair"
df_author_aff = pd.read_sql(sql = sql_cmd, con = engine)

In [103]:
df_1 = df_affname.loc[df_affname['affiliation-name'] == 'Univ of']

In [104]:
df_1

,aff_id,affiliation-name
3333,100316017,Univ of
7492,100311627,Univ of


In [105]:
len(df_author_aff.loc[df_author_aff['aff_id'] == '101516079' ]) + len(df_author_aff.loc[df_author_aff['aff_id'] == '100316017' ])

3

In [61]:
df_affname.loc[df_affname['affiliation-name'] == 'V.Associates']

,aff_id,affiliation-name
2025,101516079,V.Associates


In [106]:
len(df_author_aff.loc[df_author_aff['aff_id'] == '101516079' ])

2

In [62]:
df_affname.loc[df_affname['affiliation-name'] == 'University of']

,aff_id,affiliation-name
6216,100833241,University of
7646,105401386,University of
8062,106216222,University of
8484,113285648,University of


In [107]:
len(df_author_aff.loc[df_author_aff['aff_id'] == '100833241' ]) + len(df_author_aff.loc[df_author_aff['aff_id'] == '105401386' ]) + len(df_author_aff.loc[df_author_aff['aff_id'] == '106216222' ]) + len(df_author_aff.loc[df_author_aff['aff_id'] == '113285648' ]) 

5

In [63]:
df_affname.loc[df_affname['affiliation-name'] == 'Univ. B.']

,aff_id,affiliation-name
3500,100441329,Univ. B.
7530,115765308,Univ. B.


In [108]:
len(df_author_aff.loc[df_author_aff['aff_id'] == '100441329' ]) + len(df_author_aff.loc[df_author_aff['aff_id'] == '115765308' ])

2

In [109]:
df_affname.loc[df_affname['affiliation-name'] == 'MMU']

,aff_id,affiliation-name
2475,109892807,MMU
8690,109964508,MMU


In [110]:
len(df_author_aff.loc[df_author_aff['aff_id'] == '109892807' ]) + len(df_author_aff.loc[df_author_aff['aff_id'] == '109964508' ])

8

In [111]:
df_affname.loc[df_affname['affiliation-name'] == 'MIS']

,aff_id,affiliation-name
1155,100598022,MIS
1563,105333290,MIS
2723,105145939,MIS
3497,100529704,MIS
7290,105768363,MIS
8524,100393083,MIS


In [113]:
len(df_author_aff.loc[df_author_aff['aff_id'] == '100598022' ]) + len(df_author_aff.loc[df_author_aff['aff_id'] == '105333290' ]) +len(df_author_aff.loc[df_author_aff['aff_id'] == '105145939' ]) + len(df_author_aff.loc[df_author_aff['aff_id'] == '100529704' ]) +len(df_author_aff.loc[df_author_aff['aff_id'] == '105768363' ]) + len(df_author_aff.loc[df_author_aff['aff_id'] == '100393083' ])

9

In [112]:
df_affname.loc[df_affname['affiliation-name'] == 'CEO']

,aff_id,affiliation-name
6580,116074159,CEO
6751,108918599,CEO


In [114]:
len(df_author_aff.loc[df_author_aff['aff_id'] == '116074159' ]) + len(df_author_aff.loc[df_author_aff['aff_id'] == '108918599' ])

2